# Análisis de imágen con Python

In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from sklearn.cluster import KMeans
from scipy import ndimage
import skimage as skim
from skimage import color
from skimage import io
from skimage.color import label2rgb
from skimage.measure import regionprops
import pandas as pd
import glob
import os

In [4]:
archivos = glob.glob('0 trehalosa/*.jpg')

nueva_carpeta = 'img_procesadas'  # Carpeta donde se van a guardar las imagenes procesadas por el script
if not os.path.exists(nueva_carpeta):
    os.makedirs(nueva_carpeta)
    
nombres = []
i = 0
for archivo in archivos:
    file = os.path.basename(archivos[i])
    file1 = file.replace('.jpg','')
    nombres.append(file1)
    i = i+1
df = pd.DataFrame(columns = ['Muestra', 'L', 'a', 'b'])  # DataFrame donde se almacenarán los valores de color
j=0
for archivo in archivos:
    img = skim.io.imread(archivo)  # abro el archivo de imagen
    img_lab = color.rgb2lab(img, illuminant='D65', observer='2')  # img_Lab es un array de numpy de 3 dimensiones (L, a, b)
    ab = img_lab[:,:,1:3]  # Extraigo las componentes a y b, correspondientes a las matrices :,:,1 y :,:,2
    nrows = ab.shape[0]  # Cuento el número de filas
    ncols = ab.shape[1]  # Cuento el número de columnas
    ab = np.reshape(ab,[nrows*ncols,2])  # reorganizo la matriz ab de manera de tener una matriz en 1-D
    kmeans = KMeans(n_clusters=2, random_state=0).fit(ab)  # kmeans para separar dos poblaciones (el color del fondo y el color de las muestras)
    pixel_labels = np.reshape(kmeans.labels_,[nrows,ncols])  # En esta matriz están los labels de cada pixel etiquetado por kmeans  
    rgb_label = np.repeat(pixel_labels[:, :, np.newaxis], 3, axis=2)  # Label en 3-D para poder aplicarlo a la imagen original
    img_clean = img * rgb_label  # Multiplicando la imagen original por rgb_label del kmeans, elimino el fondo ya que pertence a la poblacion identificada con 0
    labeled, nr_objects = ndimage.label(img_clean[:,:,0])  # Etiquetado de la imagen por el canal R
    imagenes = {}  # Diccionario donde se almacenará cada objeto identificado y recortado
    i = 0  # Valor incial del contador i
    for region in regionprops(labeled):  # Loop que itera en cada region identificada mediante regionprops en la imagen etiquetada anteriormente
        if region.area >= 100:  # Condición para considerar regiones grandes, sirve como filtro
            y1, x1, y2, x2 = region.bbox  # Mediante el método bbox se extrae la información correspondiente a la posición de las regiones
            imagenes[i] = img_clean[y1:y2, x1:x2, :]  # Recorta el objeto identificado según las coordenadas de la región
            i = i + 1  # Actualiza el valor del contador
    fileName1 = 'img_procesadas/%s_1.png' % nombres[j]
    fileName2 = 'img_procesadas/%s_2.png' % nombres[j]
    fileName3 = 'img_procesadas/%s_3.png' % nombres[j]
            
    fotos = {'imagen1': skim.img_as_ubyte(imagenes[0]),
             'imagen2': skim.img_as_ubyte(imagenes[1]),
             'imagen3': skim.img_as_ubyte(imagenes[2])};
    skim.io.imsave(fileName1,fotos['imagen1'])
    skim.io.imsave(fileName2,fotos['imagen2'])
    skim.io.imsave(fileName3,fotos['imagen3'])
    objetos_lab = {}  # Diccionario donde se guardan las imagenes convertidas a L a b
    k = 0  # Valor incial del contador i
    for imagen in fotos:
        objetos_lab[k] = color.rgb2lab(fotos[imagen], illuminant='D65', observer='2')  # Convierte cada imagen a L a b
        L = np.round(np.nanmean(np.where(objetos_lab[k][:,:,0]!=0,objetos_lab[k][:,:,0],np.nan)),decimals=2)  # Calcula el promedio de L en la primer matriz sin tener en cuenta los 0
        a = np.round(np.nanmean(np.where(objetos_lab[k][:,:,1]!=0,objetos_lab[k][:,:,1],np.nan)),decimals=2)  # porque los convierte en NaN y la función promedio no los considera
        b = np.round(np.nanmean(np.where(objetos_lab[k][:,:,2]!=0,objetos_lab[k][:,:,2],np.nan)),decimals=2)  # además redondea los valores obtenidos a 2 decimales
        nueva_fila = {'Muestra':nombres[j], 'L':L, 'a':a, 'b':b} # Diccionario donde almacena los valores calculados en cada Loop
        df = df.append(nueva_fila, ignore_index=True)  # Agrega los valores calculados al DataFrame inicial
        k = k + 1
    j = j + 1
df.to_excel('tabla_color.xlsx',index=False)  # Guarda la tabla de colores en un archivo excel
df

/usr/local/lib/python3.7/site-packages/skimage/util/dtype.py:503: UserWarning: Downcasting int32 to uint8 without scaling because max value 255 fits in uint8
  return convert(image, np.uint8, force_copy)
/usr/local/lib/python3.7/site-packages/skimage/util/dtype.py:503: UserWarning: Downcasting int32 to uint8 without scaling because max value 255 fits in uint8
  return convert(image, np.uint8, force_copy)


,Muestra,L,a,b
0,melaessiccata 1.5h,71.02,17.83,27.67
1,melaessiccata 1.5h,67.63,17.09,25.91
2,melaessiccata 1.5h,70.56,17.23,27.94
3,melaessiccata 0.5h,67.80,19.85,24.05
4,melaessiccata 0.5h,65.01,21.23,23.84
5,melaessiccata 0.5h,72.38,20.26,24.82
